# **불용어 제거 및 표준화**
- 한 행 씩 보면서, 불용어 표준화 진행을 위한 전처리를 먼저 수행

In [1]:
import re
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from konlpy.tag import Okt
from ckonlpy.tag import Twitter as Okt
from ckonlpy.tag import Postprocessor
from collections import Counter
from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

logging.getLogger('konlpy').setLevel(logging.ERROR)

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

code_path = '/VSC/JJaemni/Code/'
csv_path = '/VSC/JJaemni/CSV/'

df = pd.read_csv(csv_path + 'wanted_jobplanet_2.csv')



# 리스트 형태로 불러오기
def text_list_load(text_path):
    with open(text_path, 'r', encoding='utf-8-sig') as file:
        L = [line.strip() for line in file.readlines()]
    return L

# 딕셔너리 형태로 불러오기
def text_dict_load(text_path):
    L = dict()
    with open(text_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            key, val = line.strip().split(': ')
            L[key] = val
    return L



# 한글 전처리
def Korean_preprocessing(col):
    col_list = df[col].tolist()
    
    # 한글, 영문 대소문자, 숫자, 공백을 제외한 문자를 제거
    col_list2 = []
    for idx in range(0, df.shape[0]):
        str = col_list[idx]
        new_str = re.sub('\n', '', str)
        new_str2 = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s]', ' ', new_str)
        new_str3 = re.split(' ', new_str2)
        new_str4 = ' '.join(filter(None, new_str3))
        col_list2.append(new_str4)
        # print(f'{idx}. {col_list2[idx]}')
    col_list2_df = pd.DataFrame({'col_list': col_list2})


    # Kor, Eng 분리
    eng_list = []
    kor_list = []
    for idx in range(0, df.shape[0]):
        text = col_list2_df.col_list[idx]
        eng_text = ' '.join(re.findall(r'[A-Za-z]+', text))
        kor_text = re.sub(r'[A-Za-z]+', '', text).strip()
        eng_list.append(eng_text)
        kor_list.append(kor_text)


    # 영어 띄어쓰기 기준으로 split
    eng_list_spl = []
    for eng in eng_list:
        spl = eng.split()
        eng_list_spl.append(spl)
    # for idx, eng in enumerate(eng_list_spl):
        # print(f'{idx}. {eng}')


    # 영어 소문자로 통일
    eng_lower = []
    for sub in eng_list_spl:
        sublist = []
        for eng in sub:
            eng_ = eng.lower()
            sublist.append(eng_)
        eng_lower.append(sublist)
    # for idx, eng in enumerate(eng_lower):
        # print(f'{idx}. {eng}')


    # 텍스트 파일 불러오기
    text_path = './custom_nouns.txt'
    custom_nouns = text_list_load(text_path)
    # print(custom_nouns)
    text_path = './replace.txt'
    replace = text_dict_load(text_path)
    # print(replace)
    text_path = './stopwords.txt'
    stopwords = text_list_load(text_path)
    # print(stopwords)


    # Okt 시작
    okt = Okt()


    # 명사 추가
    custom_nouns = custom_nouns
    for noun in custom_nouns:
        okt.add_dictionary(noun, 'Noun')


    # 단어 치환
    replace = replace


    # 불용어 처리
    stopwords = stopwords
    postprocessor = Postprocessor(
        base_tagger=okt,
        passtags={'Noun'},
        replace=replace,
        stopwords=stopwords,
        # passwords=passwords,
        # ngrams=ngrams
    )


    # Postprocessor 처리 후, 한 글자인 단어 제거
    results = []
    for idx in range(0, df.shape[0]):
        elem = kor_list[idx]
        # print(postprocessor.pos(elem))
        post_onechar = postprocessor.pos(elem)
        result = []
        for word, tag in post_onechar:
            if len(word) > 1:
                result.append(word)
        results.append(result)
        # print(f'{idx}. {result}')


    # 리스트 중복 요소 제거
    kor_final = []
    for idx in range(0, df.shape[0]):
        a = set(results[idx])
        b = list(a)
        kor_final.append(b)
        print(f'{idx}. {b}')
    # print(len(kor_final))


    # 데이터프레임 생성
    kor_final_df = pd.DataFrame({f'{col}_KOR': kor_final})
    kor_final_df[f'{col}_KOR'] = kor_final_df[f'{col}_KOR'].apply(lambda x: ', '.join(x))
    # kor_final_df.to_csv(csv_path + f'{col}_KOR.csv', index=False)

    return col_list2, col_list2_df, eng_list, kor_list, eng_list_spl, eng_lower, custom_nouns, replace, stopwords, results, kor_final, kor_final_df

### **Requirement_KOR**

In [2]:
col = 'Requirement'
(col_list2, col_list2_df, 
 eng_list, kor_list, 
 eng_list_spl, eng_lower, 
 custom_nouns, replace, stopwords, 
 results, kor_final, kor_final_df) = Korean_preprocessing(col)

df.insert(4, 'Requirement_KOR', kor_final_df)

c:\Users\LG\anaconda3\envs\crawling\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


0. ['데이터 분석', '인공지능', '통계분석', '알고리즘 개발', '시계열 데이터', '논문 구현', '머신러닝']
1. ['블록체인', '데이터 분석', '온체인 데이터', '데이터 사이언스', '머신러닝']
2. ['데이터 분석', '블록체인', '실무경험', '문제 정의 및 해결', '데이터 사이언스', '알고리즘 개발', '논문 구현', '머신러닝']
3. ['데이터 분석', '고객 분석', '개발역량', '상용 서비스 구현', '데이터 모델링', '인사이트 도출']
4. ['데이터 모델링']
5. ['데이터 분석', '적극적', '프레임워크', '커뮤니케이션', '협업', '주도적', '데이터 시각화', '모델 개발 및 배포', '프로젝트 경험', '모델 개발', '딥러닝', '포트폴리오', '개발경험']
6. ['고객 분석', '커뮤니케이션', '협업', '실무경험', '수리적 지식', '프로그래밍 언어', '개인화 추천', '도메인 지식', '프로젝트 경험', '데이터 모델링', '데이터 사이언티스트']
7. ['데이터 분석', '자연어 처리', '산업공학', '통계분석', '최적화', '알고리즘 개발', '도메인 지식', '컴퓨터공학', '데이터 모델링', '가설 수립 및 검증', '모델 개발']
8. ['데이터 분석', '주도적', '실무경험', '문제 정의 및 해결', '모바일 데이터', '끈질김', '프로젝트 경험', '인사이트 도출', '머신러닝', '이슈 해결']
9. ['데이터 분석', '열정적', '산업공학', '데이터 시각화', '최신 기술 연구', '통계분석', '컴퓨터공학']
10. ['커뮤니케이션', '협업', '실무경험', '문제 정의 및 해결', '모델 개발', '프로젝트 경험', '논문 구현', '머신러닝', '리딩 능력']
11. ['데이터 분석', '데이터 추출', '데이터 마트 구축 운영', '데이터 핸들링', '프로젝트 경험', '모델 개발', '데이터 전처리']
12. ['데이터 분석', '커뮤니케이션',

### **Requirement_ENG**

In [3]:
print(eng_lower)

[['python'], ['sql', 'hql', 'python', 'r'], ['sql', 'hql', 'python', 'r', 'complex', 'network', 'modeling'], ['machine', 'learnig', 'python', 'java', 'sql', 'r'], [], ['computer', 'vision', 'image', 'video', 'processing', 'tensorflow', 'pytorch', 'or'], ['ml', 'dl', 'python', 'sql', 'ml', 'dl'], ['data', 'science', 'ml', 'dl', 'nlp', 'clustering', 'ml', 'dl', 'sql', 'python'], ['python', 'sql', 'r'], ['data', 'science', 'python', 'r', 'sql', 'tool', 'splunk', 'tableau', 'tool'], ['ai'], ['python', 'data', 'analysis', 'process', 'eda', 'statistical', 'analysis', 'data', 'mining', 'report', 'deployment', 'query', 'manipulation', 'r'], ['python', 'sql', 'spark', 'hadoop'], ['ai'], [], ['python', 'r', 'ml', 'dl'], [], ['master', 's', 'degree', 'in', 'a', 'quantitative', 'discipline', 'e', 'g', 'statistics', 'operations', 'research', 'bioinformatics', 'economics', 'computational', 'biology', 'computer', 'science', 'mathematics', 'physics', 'electrical', 'engineering', 'industrial', 'enginee

In [5]:
text_path = './custom_nouns_stacktool.txt'
custom_nouns = text_list_load(text_path)

text_path = './stopwords_stacktool_eng.txt'
stopwords = text_list_load(text_path)

okt = Okt()

custom_nouns = custom_nouns
for noun in custom_nouns:
    okt.add_dictionary(noun, 'Noun')

postprocessor = Postprocessor(
    base_tagger=okt,
    passtags={'Noun'},
    # replace=replace,
    stopwords=stopwords,
    # passwords=passwords,
    # ngrams=ngrams
)

results = []
for idx in range(0, df.shape[0]):
    elem = eng_lower[idx]
    # print(postprocessor.pos(elem))
    post_onechar = postprocessor.pos(elem)
    result = []
    for word, tag in post_onechar:
        result.append(word)
    results.append(result)
    # print(f'{idx}. {result}')

eng_final = []
for idx in range(0, df.shape[0]):
    a = set(results[idx])
    b = list(a)
    eng_final.append(b)
    print(f'{idx}. {b}')
# print(len(kor_final))

c:\Users\LG\anaconda3\envs\crawling\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


AttributeError: 'list' object has no attribute 'split'

### **Preference_KOR**

In [ ]:
col = 'Preference'
(col_list2, col_list2_df, 
 eng_list, kor_list, 
 eng_list_spl, eng_lower, 
 custom_nouns, replace, stopwords, 
 results, kor_final, kor_final_df) = Korean_preprocessing(col)

df.insert(6, 'Preference_KOR', kor_final_df)

### **Preference_ENG**

In [ ]:
df.head()

### **WordCloud**

In [ ]:
def wordcloud_show(list):
    join_test1 = ', '.join(list)
    test1_list = [join_test1]
    # print(test1_list)

    font_path = '/Users/LG/AppData/Local/Microsoft/Windows/Fonts/BMJUA_ttf.ttf'
    test1_wc = ', '.join(test1_list)
    test1_wc = test1_wc.replace(' ', '').replace(',', ', ')

    wordcloud = WordCloud(
        width=2500, 
        height=2500, 
        background_color='white',
        # colormap='seismic',
        font_path=font_path
    ).generate(test1_wc)
    plt.figure(figsize=(6, 6), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.show()

# DS
# Requirement
print('DS: Requirement')
test1 = df.loc[df.label == '데이터 사이언티스트'].iloc[:, 4].tolist()
wordcloud_show(test1)
# Preference
print('DS: Preference')
test1 = df.loc[df.label == '데이터 사이언티스트'].iloc[:, 6].tolist()
wordcloud_show(test1)

# DA
# Requirement
print('DA: Requirement')
test1 = df.loc[df.label == '데이터 애널리스트'].iloc[:, 4].tolist()
wordcloud_show(test1)
# Preference
print('DA: Preference')
test1 = df.loc[df.label == '데이터 애널리스트'].iloc[:, 6].tolist()
wordcloud_show(test1)

# DE
# Requirement
print('DE: Requirement')
test1 = df.loc[df.label == '데이터 엔지니어'].iloc[:, 4].tolist()
wordcloud_show(test1)
# Preference
print('DE: Preference')
test1 = df.loc[df.label == '데이터 엔지니어'].iloc[:, 6].tolist()
wordcloud_show(test1)

### **StackTool**

- 회의 결과, `StackTool`에서의 한글은 `Requirement_KOR`, `Preference_KOR`에도 있는 내용이어서 제거하기로 함
- `StackTool`은 한글을 지운 후에 남는 영어로 정리하기
- `Requirement`와 `Preference`에 있는 영어엔, `StackTool`의 영어를 불용어 사전으로 적용하여 전처리하기

In [ ]:
stack_list = df.StackTool.tolist()

# 개행문자와 쉼표 제거 후, 쉼표로 join
stack_list2 = []
for data in stack_list:
    new_str = data.replace('\n', ' ').replace(',', ' ')
    new_str2 = new_str.split()
    new_str3 = ', '.join(new_str2)
    stack_list2.append(new_str3)
print(stack_list2)

# 하나의 리스트로 합치기
stack_merge = []
for data in stack_list2:
    spl = data.split(', ')
    stack_merge.extend(spl)
print(stack_merge)

In [ ]:
# Kor, Eng 분리
kor_list = []
eng_list = []
for data in stack_merge:
    kor_match = re.findall(r'[가-힣]+', data)
    eng_match = re.findall(r'[a-zA-Z0-9.!@#$%^&*()+]+', data)
    kor_list.extend(kor_match)
    eng_list.extend(eng_match)
print(kor_list)
print(eng_list)

In [ ]:
# 영어 소문자로 통일
eng_lower = []
for eng in eng_list:
    eng_low = eng.lower()
    eng_lower.append(eng_low)

# 중복 요소 제거
eng_list_remove = list(set(eng_lower))
print(eng_list_remove)
kor_list_remove = list(set(kor_list))
print(kor_list_remove)

# 텍스트 저장
text_path = './stopwords_stacktool_eng.txt'
with open(text_path, 'w', encoding='utf-8-sig') as file:
    for data in eng_list_remove:
        file.write(data + '\n')
text_path = './stopwords_stacktool_kor.txt'
with open(text_path, 'w', encoding='utf-8-sig') as file:
    for data in kor_list_remove:
        file.write(data + '\n')

In [ ]:
# stack_list2 소문자로 통일
stack_list2_lower = []
for data in stack_list2:
    data_low = data.lower()
    stack_list2_lower.append(data_low)
print(stack_list2_lower)

In [ ]:
# 텍스트 파일 불러오기
text_path = './stopwords_stacktool_kor.txt'
stopwords = text_list_load(text_path)
# print(stopwords)
text_path = './custom_nouns_stacktool.txt'
custom_nouns = text_list_load(text_path)
# print(custom_nouns)

okt = Okt()

custom_nouns = custom_nouns
for noun in custom_nouns:
    okt.add_dictionary(noun, 'Noun')

postprocessor = Postprocessor(
    base_tagger=okt,
    passtags={'Noun'},
    # replace=replace,
    stopwords=stopwords,
    # passwords=passwords,
    # ngrams=ngrams
)

results = []
for idx in range(0, df.shape[0]):
    elem = stack_list2_lower[idx]
    # print(postprocessor.pos(elem))
    post_onechar = postprocessor.pos(elem)
    result = []
    for word, tag in post_onechar:
        result.append(word)
    results.append(result)
    print(f'{idx}. {result}')

In [ ]:
# 데이터프레임 생성
eng_final_df = pd.DataFrame({'StackTool_ENG': results})
eng_final_df['StackTool_ENG'] = eng_final_df['StackTool_ENG'].apply(lambda x: ', '.join(x))
eng_final_df.to_csv(csv_path + 'StackTool_ENG.csv', index=False)

# insert
df.insert(8, 'StackTool_ENG', eng_final_df)

In [ ]:
df.head()

In [ ]:
# idx = df[df.StackTool_ENG == ''].index
# for i in idx:
#     print(f'{i}. {df.StackTool[i]}')

- Requirement 쪽으로 가기
    - Requirement, Preference 영어 전처리 해야 됨